In [ ]:
import xarray
import numpy as np
import sys

In [ ]:
import scipy
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
def subsample(data, sample_size):
    samples = list(zip(*[iter(data)]*sample_size))   # use 3 for triplets, etc.
    return map(lambda x:sum(x)/float(len(x)), samples)

In [ ]:
cyclone_ds = xarray.open_dataset("/g/data/x77/ob2720/partition/train/1999013S13164.nc", \
                                         engine='netcdf4', cache=True, chunks='auto', decode_cf=False)
cyclone_ds

In [ ]:
cyclone_ds = xarray.open_dataset("/g/data/x77/ob2720/partition/train/1999031N08092.nc", \
                                         engine='netcdf4', cache=True, chunks='auto', decode_cf=False)
cyclone_ds = cyclone_ds[dict(time=list(range(0, 8)),level=[1])][['v']]
cyclone_array = cyclone_ds.to_array().to_numpy()
cyclone_array = cyclone_array[:,:,:,40:120,40:120]
cyclone_array_new = cyclone_array[:,:,:,::4,::4].copy()
cyclone_array = cyclone_array.transpose((1,0,2,3,4))
cyclone_array_new = cyclone_array_new.transpose((1,0,2,3,4))
cyclone_array = cyclone_array.reshape((8,-1,80,80))
cyclone_array_new = cyclone_array_new.reshape((8,-1,20,20))
# print(cyclone_array[0])
print(cyclone_array_new[0])

In [ ]:
array = np.load("/g/data/x77/jm0124/np_cyclones_crop/4/train/1979066S15124-41.npy")
print(array[0])

In [ ]:
import seaborn
seaborn.heatmap(array[1,0,:,:]/1000)

In [ ]:
!ls /g/data/x77/jm0124/np_cyclones_crop/2/train

In [ ]:
print(array[::-1][0])
print(array[-1])

In [ ]:
from datasets import *

In [ ]:
!ls /g/data/x77/ob2720/partition/train/